# K-최근접 이웃 (K-Nearest Neighbors) 알고리즘 

- k-최근접 이웃은 이해하기 쉽고 매우 효과적인 알고리즘
- 장점 : 높은 정확도, outlier에 둔감
- 단점 : 계산 비용이 높음, 많은 메모리를 요구
- 적용 : 수치형, 명목형 

## k-최근접 이웃의 동작
- 기존에 훈련 집합이었던 데이터 집합이 있을때, 모든 데이터는 분류 항목(labels)이 붙어있기 때문에 각각의 데이터가 어떤 분류 항목으로 구분되는지 알 수 있다. 
- 이후 분류 항목 표시(label)가 붙어 있지 않은 새로운 데이터가 주어졌을 때, 기존의 모든 데이터와 새로운 데이터를 비교한다. 
- 그리고 가장 유사한 데이터들의 분류 항목을 확인한다.
- 이때, 분류 항목을 이미 알고 있는 데이터 집합에서 상위 k개의 가장 유사한 데이터를 살표본다.
- 마지막으로, k 개의 가장 유사한 데이터들 중 majority vote를 통해 새로운 데이터의 분류 항목을 결정하게 된다.

<img src="img/knn.jpg"/>

## 1. 로맨스 영화와 액션 영화 분류 예제 

- 많은 영화를 관람한 사람에 의해 저장된 데이터
- 각각의 영화마다 발차기 장면과 키스 장면의 출현 횟수를 센다. 
- 만약 관람하지 않은 영화가 있을 때, 그 영화가 액션 영화인지 로맨스 영화인지 알기위해 k-NN 알고리즘 사용

<img src="img/fig1.jpg"/>

<img src="img/fig2.jpg"/>

<img src="img/fig3.jpg"/>

## 2. kNN 분류 알고리즘 구현 및 실행

In [1]:
from numpy import *
import operator

def createDataSet():
    group = array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]]) # trainingSet
    labels = ['A','A','B','B'] # labels
    return group, labels

In [2]:
group, labels = createDataSet()
print (group)
print (labels)

[[1.  1.1]
 [1.  1. ]
 [0.  0. ]
 [0.  0.1]]
['A', 'A', 'B', 'B']


In [11]:
from numpy import *
import operator

def classify0(inX, dataSet, labels, k):
    
    dataSetSize = dataSet.shape[0] # dataSet의 전체 길이
    # 유클리디안 거리 계산
    diffMat = tile(inX, (dataSetSize,1)) - dataSet # dataSet과 inX의 차를 계산
    sqDiffMat = diffMat**2 # 제곱 연산
    sqDistances = sqDiffMat.sum(axis=1) # 두 데이터의 합을 계산
    distances = sqDistances**0.5 # 루트 연산
    # 오름차순 정렬
    sortedDistIndicies = distances.argsort()
    
    classCount={}
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1 # a = a + 1
    
    sortedClassCount = sorted(classCount.iteritems(),
                             key = operator.itemgetter(1), reverse=True) # 내림차순 정렬
    
    return sortedClassCount[0][0]

In [12]:
classify0([0,0], group, labels, 3)

'B'

## 3. 필기체 인식 시스템

- 사용 데이터 : Optical Recognition of Handwritten Digits Data Set
- 0 ~ 9 까지의 번호를 사용 예제는 아래 그림과 같음
- 번호는 이미지 처리 소프트웨어를 통해 처리됨
- 32x32 검은색(1)과 흰색(0)으로 되어있음.

<img src="img/fig4.jpg"/>

In [13]:
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

In [24]:
testVector = img2vector('digits/testDigits/0_13.txt')
print shape(testVector)
print testVector[0,0:31]

(1L, 1024L)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


### 3.1 필기체 검사에 kNN 적용하기

In [36]:
from os import listdir

def handwritingClassTest():
    hwLabels = []
    trainingFileList = listdir('digits/trainingDigits')           #load the training set
    m = len(trainingFileList) # training 파일 전체 갯수 
    trainingMat = zeros((m,1024)) # 각 파일에 있는 데이터를 저장하기 위한 매트릭스
    for i in range(m):
        # 파일 이름에서 라벨을 떼어내기 위한 작업 
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        
        # 각 파일에서 데이터를 읽어옴
        trainingMat[i,:] = img2vector('digits/trainingDigits/%s' % fileNameStr)
   
    testFileList = listdir('digits/testDigits')        #iterate through the test set
    errorCount = 0.0
    mTest = len(testFileList) # test 파일의 전체 개수
    for i in range(mTest):
        # 파일 이름에서 라벨을 떼어내기 위한 작업
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        
        # 파일에서 데이터를 읽어옴
        vectorUnderTest = img2vector('digits/testDigits/%s' % fileNameStr)
        
        # kNN 동작
        classifierResult = classify0(vectorUnderTest, trainingMat, hwLabels, 3)
        print "the classifier came back with: %d, the real answer is: %d" % (classifierResult, classNumStr)
        
        # 실제 라벨과 예측결과 라벨이 다른 경우 errorcount 증가 
        if (classifierResult != classNumStr): errorCount += 1.0
    
    print "\nthe total number of errors is: %d" % errorCount
    print "\nthe total error rate is: %f" % (errorCount/float(mTest))

In [33]:
handwritingClassTest()

the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answe

the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answe

the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answe

the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answe

the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answe

the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 6, the real answer is: 8
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 8, the real answe

the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 7, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answe